In [33]:
import pandas as pd
import requests
import json
pd.options.mode.copy_on_write = True
url = "https://api.cartola.globo.com/atletas/mercado"
response = requests.get(url)
json_data = response.json()
data_dict = json.loads(response.text)
atletas_df = pd.json_normalize(data_dict["atletas"],max_level=2)
atletas_df=atletas_df.add_prefix('atleta_')
print(atletas_df.columns.tolist())
atletas_df = atletas_df.rename(columns={'atleta_atleta_id':'atleta_id',
                                        'atleta_clube_id': 'clube_id',
                                        'atleta_posicao_id':'posicao_id',
                                        'atleta_status_id':'status_id'})

# Create the clubes DataFrame
clubes_df = pd.DataFrame.from_dict(data_dict["clubes"], orient='index')
clubes_df=clubes_df.add_prefix('clube_')
print(clubes_df.columns.tolist())

# Create the posicoes DataFrame
posicoes_df = pd.DataFrame.from_dict(data_dict["posicoes"], orient='index')
posicoes_df=posicoes_df.add_prefix('posicao_')
print(posicoes_df.columns.tolist())

#Create the status dataframe
status_df=pd.DataFrame.from_dict(data_dict["status"], orient='index')
status_df=status_df.add_prefix('status_')
print(status_df.columns.tolist())

colunas = ['atleta_apelido','clube_abreviacao','posicao_abreviacao','status_nome', 'atleta_preco_num']

partidas_response = requests.get('https://api.cartola.globo.com/partidas')
partidas_json_data = partidas_response.json()
partidas_df = pd.json_normalize(partidas_json_data["partidas"],max_level=1)
print(partidas_df.columns.tolist())
confrontos_df = partidas_df.merge(clubes_df.add_prefix('casa_'), left_on='clube_casa_id', right_on='casa_clube_id')
confrontos_df = confrontos_df.merge(clubes_df.add_prefix('visitante_'), left_on='clube_visitante_id', right_on='visitante_clube_id')
confrontos_df['confronto_slug'] = confrontos_df['casa_clube_abreviacao'] + ' X ' + confrontos_df['visitante_clube_abreviacao']
print(confrontos_df.columns.tolist())

['atleta_minimo_para_valorizar', 'atleta_jogos_num', 'atleta_atleta_id', 'atleta_rodada_id', 'atleta_clube_id', 'atleta_posicao_id', 'atleta_status_id', 'atleta_pontos_num', 'atleta_media_num', 'atleta_variacao_num', 'atleta_preco_num', 'atleta_entrou_em_campo', 'atleta_slug', 'atleta_apelido', 'atleta_apelido_abreviado', 'atleta_nome', 'atleta_foto', 'atleta_scout.V', 'atleta_gato_mestre.scouts.media', 'atleta_gato_mestre.scouts.mandante', 'atleta_gato_mestre.scouts.visitante', 'atleta_gato_mestre.media_pontos_mandante', 'atleta_gato_mestre.media_pontos_visitante', 'atleta_gato_mestre.media_minutos_jogados', 'atleta_gato_mestre.minutos_jogados', 'atleta_scout.DE', 'atleta_scout.FS', 'atleta_scout.GS', 'atleta_scout.SG', 'atleta_scout.DS', 'atleta_scout.FC', 'atleta_scout.FD', 'atleta_scout.FF', 'atleta_scout.FT', 'atleta_scout.G', 'atleta_scout.I', 'atleta_scout.A', 'atleta_scout.CA', 'atleta_scout.CV', 'atleta_scout.PS', 'atleta_scout.PP', 'atleta_scout.PC', 'atleta_scout.DP', 'atlet

In [34]:
df = atletas_df.merge(posicoes_df, left_on='posicao_id',right_on='posicao_id')
df = df.merge(clubes_df, left_on='clube_id', right_on='clube_id')
df = df.merge(status_df, left_on='status_id', right_on='status_id')
df = df.merge(confrontos_df, how='left', left_on=['clube_id'], right_on=['clube_casa_id'])
df = df.merge(confrontos_df, how='left', left_on=['clube_id'], right_on=['clube_visitante_id'])
df['confronto_slug'] = df['confronto_slug_y'].fillna('') + '' + df['confronto_slug_x'].fillna('')
print(df[['confronto_slug']])

    confronto_slug
0        BOT X COR
1        FLA X VAS
2        CAP X FOR
3        FLA X VAS
4        BAH X CAM
..             ...
775      BAH X CAM
776      BAH X CAM
777      BAH X CAM
778      BAH X CAM
779      BAH X CAM

[780 rows x 1 columns]


In [35]:
df_filtrado = df[df['status_nome'] == 'Provável']
print(df_filtrado[colunas])

     atleta_apelido clube_abreviacao posicao_abreviacao status_nome  \
0        Ramón Díaz              COR                tec    Provável   
1        Léo Jardim              VAS                gol    Provável   
2         Mastriani              CAP                ata    Provável   
3             Payet              VAS                mei    Provável   
4              Hulk              CAM                ata    Provável   
..              ...              ...                ...         ...   
766      Jean Lucas              BAH                mei    Provável   
767    Rogério Ceni              BAH                tec    Provável   
770  Gabriel Xavier              BAH                zag    Provável   
772    Luciano Juba              BAH                lat    Provável   
776  Caio Alexandre              BAH                mei    Provável   

     atleta_preco_num  
0                8.16  
1                7.54  
2                7.35  
3                6.55  
4               16.63  
.. 

In [36]:
print(df_filtrado.columns.tolist())

['atleta_minimo_para_valorizar', 'atleta_jogos_num', 'atleta_id', 'atleta_rodada_id', 'clube_id', 'posicao_id', 'status_id', 'atleta_pontos_num', 'atleta_media_num', 'atleta_variacao_num', 'atleta_preco_num', 'atleta_entrou_em_campo', 'atleta_slug', 'atleta_apelido', 'atleta_apelido_abreviado', 'atleta_nome', 'atleta_foto', 'atleta_scout.V', 'atleta_gato_mestre.scouts.media', 'atleta_gato_mestre.scouts.mandante', 'atleta_gato_mestre.scouts.visitante', 'atleta_gato_mestre.media_pontos_mandante', 'atleta_gato_mestre.media_pontos_visitante', 'atleta_gato_mestre.media_minutos_jogados', 'atleta_gato_mestre.minutos_jogados', 'atleta_scout.DE', 'atleta_scout.FS', 'atleta_scout.GS', 'atleta_scout.SG', 'atleta_scout.DS', 'atleta_scout.FC', 'atleta_scout.FD', 'atleta_scout.FF', 'atleta_scout.FT', 'atleta_scout.G', 'atleta_scout.I', 'atleta_scout.A', 'atleta_scout.CA', 'atleta_scout.CV', 'atleta_scout.PS', 'atleta_scout.PP', 'atleta_scout.PC', 'atleta_scout.DP', 'atleta_scout.GC', 'posicao_nome',

In [37]:
# prompt: gere uma analises das medias de cada posicao_id onde a coluna atleta_media_num seja diferente de 0 e mostre a media de cada posicao

# Create a new dataframe with only the relevant columns
df_analise = df[['posicao_id', 'atleta_media_num','atleta_entrou_em_campo']]

# Filter out rows where atleta_media_num is 0
df_analise = df_analise[df_analise['atleta_entrou_em_campo'] == True]

# Group the dataframe by posicao_id and calculate the mean of atleta_media_num
medias_posicao = df_analise.groupby('posicao_id')['atleta_media_num'].mean()

# Print the results
print(medias_posicao)

posicao_id
1    3.529500
2    3.893200
3    3.059388
4    2.802212
5    3.580233
6    5.016500
Name: atleta_media_num, dtype: float64


In [38]:
# prompt: gere uma media dos clubes usando a coluna clube_abreviacao a media devera ser o atleta_media_num diferente de 0 ordene pela media descendente

# Create a new dataframe with only the relevant columns
df_analise = df[['clube_abreviacao', 'atleta_media_num','atleta_entrou_em_campo']]

# Filter out rows where atleta_media_num is 0
df_analise = df_analise[df_analise['atleta_entrou_em_campo'] == True]

# Group the dataframe by clube_abreviacao and calculate the mean of atleta_media_num
medias_clube = df_analise.groupby('clube_abreviacao')['atleta_media_num'].mean()

# Sort the results in descending order
medias_clube = medias_clube.sort_values(ascending=False)

# Print the results
print(medias_clube)

clube_abreviacao
PAL    4.373529
BOT    4.125882
GRE    3.840588
VAS    3.707500
FOR    3.690000
FLU    3.548824
SAO    3.520588
CRU    3.512353
JUV    3.491765
INT    3.438824
CAP    3.410588
FLA    3.268235
BAH    3.217059
COR    3.201250
CAM    3.178824
ACG    3.128235
BGT    3.120588
VIT    2.807059
CUI    2.474706
CRI    2.411176
Name: atleta_media_num, dtype: float64


# Estudos
posicao_id | posicao_nome | posicao_abreviacao
- | -| -
1 | Goleiro | gol
2 | Lateral | lat
3 | Zagueiro | zag
4 | Meia | mei
5 | Atacante | ata
6 | Técnico | tec

## Atacante

In [41]:
import numpy as np
colunas_relatorio = ['atleta_apelido','clube_abreviacao','confronto_slug','atleta_media_num','atleta_apelido','atleta_pontos_num', 'posicao_abreviacao', 'atleta_scout.G']
posicao_id = 5 #atacante
df_analise = df_filtrado[df_filtrado['posicao_id'] == posicao_id]
medias_posicao = df_analise['atleta_media_num'].mean()
df_analise['dif'] = df_analise['atleta_media_num'] - medias_posicao
df_analise = df_analise[df_analise['atleta_pontos_num'] != 0]
df_analise = df_analise[df_analise['atleta_media_num'] > medias_posicao]
df_relatorio = df_analise[colunas_relatorio]
df_relatorio = df_relatorio.sort_values(by=['atleta_pontos_num'])
print(df_relatorio)

     atleta_apelido clube_abreviacao confronto_slug  atleta_media_num  \
35           Lucero              FOR      CAP X FOR              4.79   
44          Soteldo              GRE      BGT X GRE              4.96   
748         Luciano              SAO      CRU X SAO              5.64   
687   Luiz Fernando              ACG      ACG X VIT              5.17   
12   Bruno Henrique              FLA      FLA X VAS              5.26   
752        Everaldo              BAH      BAH X CAM              4.59   
19    Luiz Henrique              BOT      BOT X COR              5.16   
529          Wesley              INT      INT X CUI              6.40   
569     Breno Lopes              FOR      CAP X FOR              5.31   
10          Vegetti              VAS      FLA X VAS              6.42   
33      Flaco López              PAL      PAL X CRI              4.66   
226           Serna              FLU      JUV X FLU              5.10   
657      Alerrandro              VIT      ACG X VIT

## Meias

In [43]:
# prompt: quero obter do dataframe_filtrado todos os jogadores que possuem atleta_media_num acima da media da atleta _posicao_id onde o atleta_variacao seja < 0 ordene pela atleta_variacao_num

posicao_id = 4 #meias
df_analise = df_filtrado[df_filtrado['posicao_id'] == posicao_id]
medias_posicao = df_analise['atleta_media_num'].mean()
df_analise['dif'] = df_analise['atleta_media_num'] - medias_posicao
df_analise = df_analise[df_analise['atleta_pontos_num'] != 0]
df_analise = df_analise[df_analise['atleta_media_num'] > medias_posicao]
df_relatorio = df_analise[colunas_relatorio]
df_relatorio = df_relatorio.sort_values(by=['atleta_pontos_num'])
print(df_relatorio)

        atleta_apelido clube_abreviacao confronto_slug  atleta_media_num  \
585          Jhon Jhon              BGT      BGT X GRE              4.67   
636    Mateus Carvalho              VAS      FLA X VAS              3.90   
53         Lucas Moura              SAO      CRU X SAO              6.30   
470       Yago Pikachu              FOR      CAP X FOR              3.46   
377     Fellipe Mateus              CRI      PAL X CRI              3.87   
479         Pochettino              FOR      CAP X FOR              5.17   
632              Erick              CAP      CAP X FOR              3.93   
55             Raniele              COR      BOT X COR              3.61   
766         Jean Lucas              BAH      BAH X CAM              4.89   
202           Monsalve              GRE      BGT X GRE              7.62   
51     Everton Ribeiro              BAH      BAH X CAM              3.99   
52          Villasanti              GRE      BGT X GRE              4.43   
317         

## Zagueiros

In [45]:
posicao_id = 3 #zagueiros
df_analise = df_filtrado[df_filtrado['posicao_id'] == posicao_id]
medias_posicao = df_analise['atleta_media_num'].mean()
df_analise['dif'] = df_analise['atleta_media_num'] - medias_posicao
df_analise = df_analise[df_analise['atleta_pontos_num'] != 0]
df_analise = df_analise[df_analise['atleta_media_num'] > medias_posicao]
df_relatorio = df_analise[colunas_relatorio]
df_relatorio = df_relatorio.sort_values(by=['atleta_pontos_num'])
print(df_relatorio)

        atleta_apelido clube_abreviacao confronto_slug  atleta_media_num  \
486            Cardona              FOR      CAP X FOR              3.30   
191      Junior Alonso              CAM      BAH X CAM              4.17   
393          Zé Ivaldo              CRU      CRU X SAO              3.82   
399       João Marcelo              CRU      CRU X SAO              3.73   
551        Danilo Boza              JUV      JUV X FLU              3.27   
658    Wagner Leonardo              VIT      ACG X VIT              3.17   
481           Kuscevic              FOR      CAP X FOR              3.61   
307             Bastos              BOT      BOT X COR              3.97   
586           Arboleda              SAO      CRU X SAO              4.11   
770     Gabriel Xavier              BAH      BAH X CAM              3.61   
30              Murilo              PAL      PAL X CRI              3.82   
132       Thiago Silva              FLU      JUV X FLU              5.94   
32       Gus

## Lateral

posicao_id = 2 #laterais
df_analise = df_filtrado[df_filtrado['posicao_id'] == posicao_id]
medias_posicao = df_analise['atleta_media_num'].mean()
df_analise['dif'] = df_analise['atleta_media_num'] - medias_posicao
df_analise = df_analise[df_analise['atleta_pontos_num'] != 0]
df_analise = df_analise[df_analise['atleta_media_num'] > medias_posicao]
df_relatorio = df_analise[colunas_relatorio]
df_relatorio = df_relatorio.sort_values(by=['atleta_pontos_num'])
print(df_relatorio)

## Goleiro

In [49]:
posicao_id = 1 #goleiros
df_analise = df_filtrado[df_filtrado['posicao_id'] == posicao_id]
medias_posicao = df_analise['atleta_media_num'].mean()
df_analise['dif'] = df_analise['atleta_media_num'] - medias_posicao
df_analise = df_analise[df_analise['atleta_pontos_num'] != 0]
df_analise = df_analise[df_analise['atleta_media_num'] > medias_posicao]
df_relatorio = df_analise[colunas_relatorio]
df_relatorio = df_relatorio.sort_values(by=['atleta_pontos_num'])
print(df_relatorio)

    atleta_apelido clube_abreviacao confronto_slug  atleta_media_num  \
338         Cássio              CRU      CRU X SAO              5.05   
762  Marcos Felipe              BAH      BAH X CAM              4.00   
171     Hugo Souza              COR      BOT X COR              3.74   
472   João Ricardo              FOR      CAP X FOR              5.08   
328        Cleiton              BGT      BGT X GRE              3.68   
610           John              BOT      BOT X COR              4.06   
490      Marchesín              GRE      BGT X GRE              4.28   
442          Fábio              FLU      JUV X FLU              4.43   
1       Léo Jardim              VAS      FLA X VAS              3.87   
556       Weverton              PAL      PAL X CRI              5.40   

    atleta_apelido  atleta_pontos_num posicao_abreviacao  atleta_scout.G  
338         Cássio                1.0                gol             NaN  
762  Marcos Felipe                1.0                gol 

## Técnicos

In [51]:
posicao_id = 6 #tecnicos
df_analise = df_filtrado[df_filtrado['posicao_id'] == posicao_id]
medias_posicao = df_analise['atleta_media_num'].mean()
df_analise['dif'] = df_analise['atleta_media_num'] - medias_posicao
df_analise = df_analise[df_analise['atleta_pontos_num'] != 0]
df_analise = df_analise[df_analise['atleta_media_num'] > medias_posicao]
df_relatorio = df_analise[colunas_relatorio]
df_relatorio = df_relatorio.sort_values(by=['atleta_pontos_num'])
print(df_relatorio)

      atleta_apelido clube_abreviacao confronto_slug  atleta_media_num  \
117    Luis Zubeldía              SAO      CRU X SAO              5.06   
424             Tite              FLA      FLA X VAS              5.53   
483     Juan Vojvoda              FOR      CAP X FOR              5.40   
279  Bernardo Franco              CUI      INT X CUI              5.38   
88   Fernando Seabra              CRU      CRU X SAO              5.36   
129     Rafael Paiva              VAS      FLA X VAS              5.87   
168     Mano Menezes              FLU      JUV X FLU              5.85   
70       Artur Jorge              BOT      BOT X COR              5.77   
574    Abel Ferreira              PAL      PAL X CRI              6.32   

      atleta_apelido  atleta_pontos_num posicao_abreviacao  atleta_scout.G  
117    Luis Zubeldía               2.04                tec             NaN  
424             Tite               2.45                tec             NaN  
483     Juan Vojvoda        